# Feature Engineering

Based on EDA findings:
- Time features
- Amount transforms 
- Category flags
- Customer age

## 1. Setup

In [88]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIR = Path('../data/interim')
PROCESSED_DIR = Path('../data/processed')
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

In [89]:
# Load data
df = pd.read_csv(DATA_DIR / 'transactions_merged.csv')

if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['dob'] = pd.to_datetime(df['dob'])

print(f"Loaded {len(df):,} rows")

Loaded 1,852,394 rows


## 2. Time Features

In [90]:
df['hour'] = df['trans_date_trans_time'].dt.hour
df['day_of_week'] = df['trans_date_trans_time'].dt.dayofweek
df['month'] = df['trans_date_trans_time'].dt.month

# Night: 22:00 - 06:00 (high fraud hours from EDA - 5x higher fraud rate)
df['is_night'] = ((df['hour'] >= 22) | (df['hour'] <= 6)).astype(int)
print(f"Night transactions: {df['is_night'].mean()*100:.1f}%")

Night transactions: 33.2%


In [91]:
# Verify
print("Fraud rate by is_night:")
print(df.groupby('is_night')['is_fraud'].mean() * 100)

Fraud rate by is_night:
is_night
0    0.104047
1    1.358999
Name: is_fraud, dtype: float64


### Findings

Night flag captures the high-fraud pattern from EDA.

## 3. Amount Features

In [92]:
# Log transform
df['log_amt'] = np.log1p(df['amt'])

print(f"Amount range: ${df['amt'].min():.2f} - ${df['amt'].max():.2f}")
print(f"Log amount range: {df['log_amt'].min():.2f} - {df['log_amt'].max():.2f}")

Amount range: $1.00 - $28948.90
Log amount range: 0.69 - 10.27


## 4. Category Features

In [93]:
# Online flag (categories ending with _net)
df['is_online'] = df['category'].str.endswith('_net').astype(int)

print("Fraud rate by is_online:")
print(df.groupby('is_online')['is_fraud'].mean() * 100)

Fraud rate by is_online:
is_online
0    0.390038
1    1.212804
Name: is_fraud, dtype: float64


In [94]:
# Merchant risk - similar approach to state risk
merchant_stats = df.groupby('merchant').agg(
    fraud_rate=('is_fraud', 'mean'),
    count=('is_fraud', 'count')
)

print(f"Total merchants: {len(merchant_stats)}")
median_merch_count = merchant_stats['count'].median()
print(f"Median merchant transactions: {median_merch_count:,.0f}")

# Top fraud merchants
print("\nTop 10 merchants by fraud rate (with >100 transactions):")
print(merchant_stats[merchant_stats['count'] > 100].sort_values('fraud_rate', ascending=False).head(10))

# Merchants below median -> 'medium', others -> quantile
low_count_mask = merchant_stats['count'] < median_merch_count
reliable_merchants = merchant_stats[~low_count_mask]
merchant_risk_reliable = pd.qcut(reliable_merchants['fraud_rate'], q=3, labels=['low', 'medium', 'high'])

merchant_risk_map = merchant_risk_reliable.to_dict()
for merch in merchant_stats[low_count_mask].index:
    merchant_risk_map[merch] = 'medium'

df['merchant_risk'] = df['merchant'].map(merchant_risk_map)

print(f"\nMerchant risk distribution:")
print(df['merchant_risk'].value_counts())

Total merchants: 693
Median merchant transactions: 2,663

Top 10 merchants by fraud rate (with >100 transactions):
                                            fraud_rate  count
merchant                                                     
fraud_Kozey-Boehm                             0.021755   2758
fraud_Herman, Treutel and Dickens             0.020321   1870
fraud_Terry-Huel                              0.019553   2864
fraud_Kerluke-Abshire                         0.018975   2635
fraud_Mosciski, Ziemann and Farrell           0.018788   2821
fraud_Schmeler, Bashirian and Price           0.018651   2788
fraud_Kuhic LLC                               0.018649   2842
fraud_Jast Ltd                                0.018498   2757
fraud_Langworth, Boehm and Gulgowski          0.018459   2817
fraud_Romaguera, Cruickshank and Greenholt    0.018432   2767

Merchant risk distribution:
merchant_risk
medium    1098250
high       381034
low        373110
Name: count, dtype: int64


## 5. Customer Features

In [95]:
# Age at transaction
df['age'] = (df['trans_date_trans_time'] - df['dob']).dt.days // 365

print(f"Age range: {df['age'].min()} - {df['age'].max()}")

Age range: 13 - 96


## 6. Geographic Features

In [96]:
# Distance (not very predictive from EDA, but include)
df['distance'] = np.sqrt(
    (df['lat'] - df['merch_lat'])**2 + 
    (df['long'] - df['merch_long'])**2
)

## 7. Risk Labels (based on internal fraud rates)

Create risk categories using fraud rates from our data. Could also use external crime data but internal rates are more relevant for this task.

In [97]:
# Category risk labels using quantiles (not hardcoded thresholds)
category_stats = df.groupby('category').agg(
    fraud_rate=('is_fraud', 'mean'),
    count=('is_fraud', 'count')
)
print("Category stats:")
print(category_stats.sort_values('fraud_rate', ascending=False))

# Use quantiles for risk levels
category_risk = pd.qcut(category_stats['fraud_rate'], q=3, labels=['low', 'medium', 'high'])
df['category_risk'] = df['category'].map(category_risk)

print("\nCategory risk distribution:")
print(df['category_risk'].value_counts())

Category stats:
                fraud_rate   count
category                          
shopping_net      0.015927  139322
misc_net          0.013039   90654
grocery_pos       0.012645  176191
shopping_pos      0.006344  166463
gas_transport     0.004106  188029
misc_pos          0.002819  114229
grocery_net       0.002697   64878
travel            0.002692   57956
personal_care     0.002229  130085
entertainment     0.002177  134118
kids_pets         0.001880  161727
food_dining       0.001568  130729
home              0.001510  175460
health_fitness    0.001510  122553

Category risk distribution:
category_risk
high      760659
low       724587
medium    367148
Name: count, dtype: int64


In [98]:
# State risk labels - handle low transaction counts
state_stats = df.groupby('state').agg(
    fraud_rate=('is_fraud', 'mean'),
    count=('is_fraud', 'count'),
    fraud_count=('is_fraud', 'sum')
)

# Use MEDIAN (not mean) because:
# - Mean is sensitive to outliers (big states like CA, TX, NY pull it up)
# - Median gives the actual middle value - more robust
median_count = state_stats['count'].median()
mean_count = state_stats['count'].mean()
print(f"Median state transactions: {median_count:,.0f}")
print(f"Mean state transactions: {mean_count:,.0f}  (higher due to big states)")

# States below median count -> set to 'medium' (unreliable fraud rate)
low_count_mask = state_stats['count'] < median_count
print(f"\nStates with < median transactions: {low_count_mask.sum()}")
print(state_stats[low_count_mask].sort_values('count').head(10))

# For states with enough data, use quantiles
reliable_states = state_stats[~low_count_mask]
state_risk_reliable = pd.qcut(reliable_states['fraud_rate'], q=3, labels=['low', 'medium', 'high'])

# Create full mapping: low-count states -> 'medium', others -> quantile-based
state_risk_map = state_risk_reliable.to_dict()
for state in state_stats[low_count_mask].index:
    state_risk_map[state] = 'medium'  # Unreliable -> neutral

df['state_risk'] = df['state'].map(state_risk_map)

print(f"\nState risk distribution:")
print(df['state_risk'].value_counts())

Median state transactions: 32,939
Mean state transactions: 36,321  (higher due to big states)

States with < median transactions: 25
       fraud_rate  count  fraud_count
state                                
DE       1.000000      9            9
RI       0.020134    745           15
AK       0.016875   2963           50
HI       0.004385   3649           16
DC       0.006043   5130           31
ID       0.004107   8035           33
NV       0.005833   8058           47
CT       0.005101  10979           56
NH       0.006737  11727           79
UT       0.003972  15357           61

State risk distribution:
state_risk
medium    844618
low       554419
high      453357
Name: count, dtype: int64


In [99]:
# Verify risk labels capture fraud pattern
print("Fraud rate by category_risk:")
print(df.groupby('category_risk')['is_fraud'].mean() * 100)

print("\nFraud rate by state_risk:")
print(df.groupby('state_risk')['is_fraud'].mean() * 100)

Fraud rate by category_risk:
category_risk
low       0.172650
medium    0.256845
high      0.980334
Name: is_fraud, dtype: float64

Fraud rate by state_risk:
state_risk
high      0.591807
low       0.456334
medium    0.525445
Name: is_fraud, dtype: float64


/tmp/ipykernel_272938/509162737.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('category_risk')['is_fraud'].mean() * 100)


## 8. Feature Encoding

- One-hot encode low cardinality: gender, category_risk, state_risk
- Label encode or drop high cardinality: category (14), state (51)

In [100]:
# One-hot encode risk labels and gender
df_encoded = pd.get_dummies(df, columns=['category_risk', 'state_risk', 'merchant_risk', 'gender'], 
                            prefix=['cat_risk', 'state_risk', 'merch_risk', 'gender'],
                            drop_first=True)

print("Encoded columns added:")
print([c for c in df_encoded.columns if 'risk' in c or 'gender' in c])

Encoded columns added:
['cat_risk_medium', 'cat_risk_high', 'state_risk_low', 'state_risk_medium', 'merch_risk_low', 'merch_risk_medium', 'gender_M']


In [101]:
# Final feature columns
feature_cols = [
    # Numerical
    'amt', 'log_amt', 'city_pop', 'age', 'distance',
    'hour', 'day_of_week', 'month',
    
    # Binary flags (based on EDA findings)
    'is_night',    # 5x higher fraud rate at night
    'is_online',   # 3x higher fraud rate online
    
    # One-hot encoded risk labels
    'cat_risk_low', 'cat_risk_medium',
    'state_risk_low', 'state_risk_medium',
    'merch_risk_low', 'merch_risk_medium',
    'gender_M'
]

# Filter existing columns
feature_cols = [c for c in feature_cols if c in df_encoded.columns]
target = 'is_fraud'

df_final = df_encoded[feature_cols + [target]].copy()
print(f"Final shape: {df_final.shape}")
print(f"\nFeatures ({len(feature_cols)}):")
for col in feature_cols:
    print(f"  - {col}")

Final shape: (1852394, 17)

Features (16):
  - amt
  - log_amt
  - city_pop
  - age
  - distance
  - hour
  - day_of_week
  - month
  - is_night
  - is_online
  - cat_risk_medium
  - state_risk_low
  - state_risk_medium
  - merch_risk_low
  - merch_risk_medium
  - gender_M


### Why these columns were excluded:

| Column | Reason |
|--------|--------|
| `trans_date_trans_time` | Extracted `hour`, `month`, `day_of_week` from it |
| `cc_num` | Identifier, not a feature |
| `merchant` | Replaced with `merchant_risk` (693 unique -> 3 risk levels) |
| `first`, `last` | Names are not predictive |
| `street`, `city`, `zip` | High cardinality, used `state_risk` instead |
| `lat`, `long`, `merch_lat`, `merch_long` | Created `distance` feature instead |
| `dob` | Created `age` feature instead |
| `trans_num` | Transaction ID |
| `unix_time` | Same info as datetime |
| `category` | Replaced with `is_online` + `category_risk` |
| `state` | Replaced with `state_risk` |

In [102]:
# Save processed data
df_final.to_parquet(PROCESSED_DIR / 'features.parquet', index=False)
print(f"Saved to {PROCESSED_DIR / 'features.parquet'}")

Saved to ../data/processed/features.parquet


## Summary

**Created 16 features from 23 original columns:**

| Feature Type | Features | Notes |
|-------------|----------|-------|
| Numerical | `amt`, `log_amt`, `city_pop`, `age`, `distance` | Log transform for skewed amount |
| Time | `hour`, `day_of_week`, `month` | Extracted from timestamp |
| Binary | `is_night`, `is_online` | Based on EDA findings |
| Risk (one-hot) | `cat_risk_*`, `state_risk_*`, `merch_risk_*` | Quantile-based risk labels |
| Demographics | `gender_M` | One-hot encoded |

**Key decisions:**
- Used quantiles (not hardcoded thresholds) for risk labels
- Low transaction count states/merchants set to 'medium' (unreliable statistics)
- Dropped high-cardinality columns in favor of risk aggregations